In [1]:
import folium
import osmnx as ox
import pandas as pd
import networkx as nx

def func(x):
    return x[0]

def func_1(x):
    gdf_1 = ox.geocode_to_gdf(x, by_osmid=True)
    return gdf_1[['lat', 'lon']]

def func_tags(tags, city):
    try:
        gdf = ox.features_from_place(city, tags)
    except ox._errors.InsufficientResponseError:
        return None
    gdf = gdf[['name', 'geometry', 'nodes']]
    gdf = gdf.dropna().reset_index()
    gdf['n_osmid'] = gdf['element_type'].apply(func) + gdf['osmid'].astype(str)
    ex = [gdf['n_osmid'].apply(func_1).iloc[:].values[x] for x in range(len(gdf))]
    dfs = pd.concat(ex, axis=0).reset_index(drop=True)
    gdf = gdf.merge(dfs, on=dfs.index)
    gdf = gdf.drop('key_0', axis=1)
    gdf = gdf[['name', 'lat', 'lon', 'geometry', 'nodes']]
    gdf['kind_of'] = list(tags.keys())[0]
    return gdf

tags_port = {'industrial' : 'port'} # port cargo
tags_aeroway = {'aeroway' : ['aerodrome','heliport', 'airstrip']}
tags_landuse = {'landuse' : 'railway'}
tags_build = {'building' : 'warehouse', 'amenity' : 'mailroom'}

# city_from = 'Saint Petersburg'
# city_to = 'Moscow'
city_from = 'Tallin'
city_to = 'Riga'

full_df_from = pd.concat([func_tags(tags_port, city_from), func_tags(tags_aeroway, city_from), func_tags(tags_landuse, city_from), func_tags(tags_build, city_from)], ignore_index=True)
full_df_to = pd.concat([func_tags(tags_port, city_to), func_tags(tags_aeroway, city_to), func_tags(tags_landuse, city_to), func_tags(tags_build, city_to)], ignore_index=True)

G_from = ox.graph_from_place(city_from, retain_all=True, simplify = True, network_type='drive')
G_to = ox.graph_from_place(city_to, retain_all=True, simplify = True, network_type='drive')

lat_from = list(full_df_from['lat'].values)
lon_from = list(full_df_from['lon'].values)

lat_to = list(full_df_to['lat'].values)
lon_to = list(full_df_to['lon'].values)

full_graph = nx.compose_all([G_from, G_to])
full_df_from['new_nodes'] = ox.distance.nearest_nodes(full_graph, lon_from, lat_from)
full_df_to['new_nodes'] = ox.distance.nearest_nodes(full_graph, lon_to, lat_to)

gdf_nodes, gdf_relationships = ox.graph_to_gdfs(full_graph)
north, south, east, west = gdf_nodes[['y']].max(), gdf_nodes[['y']].min(), gdf_nodes[['x']].min(), gdf_nodes[['x']].max()

GG = ox.graph_from_bbox(bbox = (north, south, east, west), network_type='drive')

C:\Users\orenc\AppData\Local\Temp\ipykernel_8444\71198669.py:58: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  GG = ox.graph_from_bbox(bbox = (north, south, east, west), network_type='drive')
c:\mArt\dataScience\Practice_3sem\project's\1\.venv\lib\site-packages\shapely\geometry\polygon.py:91: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(c) for c in o]


In [5]:
route = nx.shortest_path(GG, 385950102, 1668198230)

shortest_route_map = ox.plot_route_folium(GG, route, 
                                          tiles='openstreetmap')
shortest_route_map

C:\Users\orenc\AppData\Local\Temp\ipykernel_8444\1141791374.py:3: FutureWarning: The `folium` module has been deprecated and will be removed in the v2.0.0 release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  shortest_route_map = ox.plot_route_folium(GG, route,


In [ ]:
route = nx.shortest_path(GG, 22283853, 1577926006)
shortest_route_map = ox.plot_route_folium(GG, route)
shortest_route_map

In [ ]:
start_latlng = (59.887362, 30.332532)
end_latlng   = (55.58783, 37.608322)
start_location = "Склад РТИ, СПб"
end_location = "Складовка, Москва"

start_marker = folium.Marker(
            location = start_latlng,
            popup = start_location,
            icon = folium.Icon(color='green'))

end_marker = folium.Marker(
            location = end_latlng,
            popup = end_location,
            icon = folium.Icon(color='red'))

start_marker.add_to(shortest_route_map)
end_marker.add_to(shortest_route_map)
shortest_route_map